In [1]:
import os
import shutil
import random
import pandas as pd
import sys
sys.path.append('../')
from src.data import download_images

## Get Data and sort it

In [2]:
# data = pd.read_csv('data/reviewed_data_5_13_24.csv') ##We can use this for training
data = pd.read_csv('../data/final_eval_data.csv')
# data = pd.read_csv('data/justin_dataset_may_models.csv').rename(columns={'new_url': 'download_url'}) ##This we can use to bolster the R rated dataset


In [4]:
data.groupby('label')['label'].count()

label
PG       7393
PG13    13425
R        5557
X        7622
XXX      6007
Name: label, dtype: int64

In [5]:
# Count the number of rows per label
label_counts = data.groupby('label')['label'].count()

# Get the minimum count among all labels
min_count = label_counts.min()

# Sample equal number of rows per label
balanced_data = data.groupby('label').apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)

balanced_data.groupby('label')['label'].count()

/var/folders/lj/vxfv4wp1351_qdc939xvvn0m0000gn/T/ipykernel_6898/3553324668.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_data = data.groupby('label').apply(lambda x: x.sample(n=min_count, random_state=42)).reset_index(drop=True)


label
PG      5557
PG13    5557
R       5557
X       5557
XXX     5557
Name: label, dtype: int64

In [6]:
balanced_data.head()

,download_url,prompt,tags,label,datatype,mixtureModel
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,Generate a photograph of a realistic modern wo...,"beachwear, clothing, blonde, hair, person, adu...",PG,justins_eval,PG
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,"Gerudo Link, gerudolink, blonde, green see-thr...","man, blonde hair, blue eyes, crossdressing, de...",PG,justins_eval,PG13
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,photo of beautiful woman with perfect (red col...,"body freckles, nose, realistic, freckles, lips...",PG,training,NaN
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,"score_9, score_8_up, score_7_up, a red haired ...","woman, cartoon, head, face, person, publicatio...",PG,training,NaN
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,what do you want from me,"solo, strapless, realistic, parted lips, looki...",PG,training,NaN


In [15]:
data = {}

# Iterate over the DataFrame
for index, row in balanced_data.iterrows():
    data[index] = {
        "url": row['download_url'], 
        "prompt": row['prompt'],
        "tags": f"{row['tags']}",
        "label": row['label']
    }


In [16]:
output = './data/datasets/'
download_images(data, output)

KeyError: 'prompt_and_tags'

In [ ]:
for i in os.listdir(output):
    path = os.path.join(output, i)
    print(i, f"imgs: {len([i for i in os.listdir(path) if i.endswith('.jpg')])}")
    print(i, f"text: {len([i for i in os.listdir(path) if i.endswith('.txt') and 'tags' not in i])}")
    print(i, f"tags: {len([i for i in os.listdir(path) if i.endswith('.txt') and 'tags' in i])}")


PG imgs: 340
PG text: 340
PG tags: 317


## Use loop for training data

In [ ]:


# # Define the source directory containing class-specific folders
# source_dir = './data/training_data/'

# # Define the output directories for training and validation
# output_train_dir = './data/datasets/may/train'
# output_val_dir = './data/datasets/may/val'

# # Create the output directories if they don't exist
# os.makedirs(output_train_dir, exist_ok=True)
# os.makedirs(output_val_dir, exist_ok=True)

# # Iterate over each class-specific folder
# for class_name in os.listdir(source_dir):
#     class_path = os.path.join(source_dir, class_name)
    
#     if os.path.isdir(class_path):
#         # List of all .txt files (excluding those with 'tags' in their name)
#         txt_files = [f for f in os.listdir(class_path) if f.endswith('.txt') and 'tags' not in f]
        
#         num_files = len(txt_files)
#         train_split = int(num_files * 0.85)
        
#         # Create lists of indices for training and validation sets
#         shuffled_indices = list(range(num_files))
#         random.shuffle(shuffled_indices)
#         train_indices = shuffled_indices[:train_split]
#         val_indices = shuffled_indices[train_split:]

#         print(f"Class '{class_name}':")
#         print(f"  Train indices: {len(train_indices)}")
#         print(f"  Val indices: {len(val_indices)}")

#         # Copy files into 'train' and 'val' directories
#         for i in train_indices:
#             txt_file = txt_files[i]
#             img_file = txt_file.replace('.txt', '.jpg')  # Assume corresponding .jpg files
            
#             # Source paths
#             src_txt_path = os.path.join(class_path, txt_file)
#             src_img_path = os.path.join(class_path, img_file)
            
#             # Destination paths
#             dest_txt_path = os.path.join(output_train_dir, class_name, txt_file)
#             dest_img_path = os.path.join(output_train_dir, class_name, img_file)
            
#             # Copy the files
#             os.makedirs(os.path.join(output_train_dir, class_name), exist_ok=True)
#             shutil.copy2(src_txt_path, dest_txt_path)
#             if os.path.exists(src_img_path):
#                 shutil.copy2(src_img_path, dest_img_path)

#         for i in val_indices:
#             txt_file = txt_files[i]
#             img_file = txt_file.replace('.txt', '.jpg')  # Assume corresponding .jpg files
            
#             # Source paths
#             src_txt_path = os.path.join(class_path, txt_file)
#             src_img_path = os.path.join(class_path, img_file)
            
#             # Destination paths
#             dest_txt_path = os.path.join(output_val_dir, class_name, txt_file)
#             dest_img_path = os.path.join(output_val_dir, class_name, img_file)
            
#             os.makedirs(os.path.join(output_val_dir, class_name), exist_ok=True)
            
#             # Copy the files
#             shutil.copy2(src_txt_path, dest_txt_path)
#             if os.path.exists(src_img_path):
#                 shutil.copy2(src_img_path, dest_img_path)

# print("File copying complete!")

## User loop for eval

In [ ]:


# Define the source directory containing class-specific folders
print(output_dir)

# Define the output directories for training and validation
output_train_dir = './data/datasets/may/train'
output_val_dir = './data/datasets/may/val'

# Create the output directories if they don't exist
os.makedirs(output_train_dir, exist_ok=True)
os.makedirs(output_val_dir, exist_ok=True)

# Iterate over each class-specific folder
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    
    if os.path.isdir(class_path):
        # List of all .txt files (excluding those with 'tags' in their name)
        txt_files = [f for f in os.listdir(class_path) if f.endswith('.txt') and 'tags' not in f]
        
        num_files = len(txt_files)
        train_split = int(num_files * 0.85)
        
        # Create lists of indices for training and validation sets
        shuffled_indices = list(range(num_files))
        random.shuffle(shuffled_indices)
        train_indices = shuffled_indices[:train_split]
        val_indices = shuffled_indices[train_split:]

        print(f"Class '{class_name}':")
        print(f"  Train indices: {len(train_indices)}")
        print(f"  Val indices: {len(val_indices)}")

        # Copy files into 'train' and 'val' directories
        for i in train_indices:
            txt_file = txt_files[i]
            img_file = txt_file.replace('.txt', '.jpg')  # Assume corresponding .jpg files
            
            # Source paths
            src_txt_path = os.path.join(class_path, txt_file)
            src_img_path = os.path.join(class_path, img_file)
            
            # Destination paths
            dest_txt_path = os.path.join(output_train_dir, class_name, txt_file)
            dest_img_path = os.path.join(output_train_dir, class_name, img_file)
            
            # Copy the files
            os.makedirs(os.path.join(output_train_dir, class_name), exist_ok=True)
            shutil.copy2(src_txt_path, dest_txt_path)
            if os.path.exists(src_img_path):
                shutil.copy2(src_img_path, dest_img_path)

        for i in val_indices:
            txt_file = txt_files[i]
            img_file = txt_file.replace('.txt', '.jpg')  # Assume corresponding .jpg files
            
            # Source paths
            src_txt_path = os.path.join(class_path, txt_file)
            src_img_path = os.path.join(class_path, img_file)
            
            # Destination paths
            dest_txt_path = os.path.join(output_val_dir, class_name, txt_file)
            dest_img_path = os.path.join(output_val_dir, class_name, img_file)
            
            os.makedirs(os.path.join(output_val_dir, class_name), exist_ok=True)
            
            # Copy the files
            shutil.copy2(src_txt_path, dest_txt_path)
            if os.path.exists(src_img_path):
                shutil.copy2(src_img_path, dest_img_path)

print("File copying complete!")